### 협업일지
#### 일자 : 2025-04-29 / 박하진

#### 오늘 할 일
- 이탈 원인 심층 분석
- 
-

#### 오늘 한 일
-
-

#### 내일 할 일
-
-
-

#### Codes

#### Issues & Challenges
-
-
-

#### Reflection

 이탈 원인 심층 분석

목표: 특정 시점에 왜 사용자가 떠났는지를 정량/정성적으로 파악

**이탈 유저가 떠나기 전에 어떤 행동을 했는지**, 그리고 **잔존 유저와 어떤 차이가 있었는지**를 파악

1. **이탈 유저 정의 - 가입 후 7일 이내, 3일 이상 미참여 : 가입 직후 관심 없으면 이탈로 간주**

2. **비교 분석을 위한 주요 행동 변수**

| 변수명 | 설명 |
| --- | --- |
| `vote_cnt` | 해당 기간 동안 총 투표 수 |
| `unique_target_cnt` | 투표한 대상자 수 |
| `emotion_change_cnt` | 감정 추론 값 변화 횟수 |
| `positive_emotion_ratio` | 긍정 감정 비율 |
| `same_target_vote_ratio` | 같은 대상에게 반복 투표한 비율 |
| `question_diversity_score` | 참여한 질문의 주제 다양도 |
| `daily_activity_span` | 활동한 일수 (ex. 3일 중 2일 활동) |
| `last_vote_gap` | 마지막 투표 이후 지난 일수 |


변수 생성 로직을 통해 is_churned 컬럼을 만들고 `is_churned == True` 와 `False` 유저를 나눈 후 **가입 후 7일 이내 활동 일수 분포**를 통해 **이탈자와 비이탈자 비율 파악**

| active_days_in_7days  | count |
| --- | --- |
| 0.0     | 672328 |
| 7.0       | 1109 |
| 8.0        | 924 |
| 6.0        | 818 |
| 5.0        | 678 |
| 4.0        | 506 |
| 3.0        | 312 |
| 1.0        | 207 |
| 2.0        | 203 |

### **해석**

- **`0.0`일 활동 (672,328명)**: 아예 참여하지 않은 사용자 — 전형적인 이탈자
- **`1~2일` 활동**: 관심은 있었지만 꾸준한 참여 X → 이탈자 기준에 포함
- **`3일 이상` 활동**: 어느 정도는 서비스에 관심 보인 사용자 → **비이탈자**

## 이탈자 특성 분석

1. **`account_user` 테이블: 기본 사용자 프로필 정보**

| 컬럼명 | 설명 |
| --- | --- |
| id | 사용자 ID |
| gender | 성별 |
| point | 포인트 수치 |
| friend_id_list | 친구 목록 (리스트 형태) |
| is_push_on | 푸시 알림 설정 여부 |
| report_count | 신고 횟수 |
| group_id | 소속 그룹 ID |
1. 이탈 여부 (`user_churn_df`) : 가입 후 7일 이내 3일 이상 참여한 적 없는 사용자 = 이탈자(1), 그렇지 않으면 비이탈자(0)

## 1. 이탈자 vs 잔존자 특성 비교 (포인트 이상치 제거 후)

| 항목 | 이탈자 (Churned) | 잔존자 (Retained) | 요약 |
| --- | --- | --- | --- |
| **is_push_on (푸시 알림 설정)** | 0.8429 | 0.8572 | 잔존자가 푸시 알림을 약간 더 켜둠 (푸시 설정 비율이 높을수록 잔존 가능성 높음) |
| **point (포인트)** | 1,540 | 1,668 | 잔존자가 평균적으로 약간 더 많은 포인트를 가짐 (활동성 반영 가능) |
| **report_count (신고 수)** | 0.0362 | 0.0508 | 잔존자가 신고를 조금 더 많이 함 (서비스 적극적 사용 신호로 해석 가능) |
| **friend_count (친구 수)** | 53.1 | 62.8 | **잔존자가 친구 수가 평균적으로 약 10명 더 많음 (사회적 연결성 높을수록 잔존)** |
| **group_id (가입 그룹)** | 평균 37,107 | 평균 25,760 | 그룹 자체는 큰 직접적 영향은 없어 보임 (별다른 차이 없음) |

### 해석

- **푸시 알림을 켜두고**, **포인트가 더 많고**, **친구 수가 더 많고**, **신고도 약간 더 많이 하는** 사용자들이 **잔존 확률이 높음**.

→ 즉, "앱에 적극적으로 관여하는 활동성 높은 사용자"일수록 이탈을 덜 한다는 결론

## 성별 별 이탈률 해석

| 성별 | 잔존률 (%) | 이탈률 (%) |
| --- | --- | --- |
| **여성 (F)** | 0.58% | 99.42% |
| **남성 (M)** | 0.70% | 99.30% |

### 해석

- **남성**이 **조금 더 잔존율**이 높음 (0.7% vs 0.58%)
- **하지만 전체적으로 성별에 따른 이탈률 차이는 거의 없음**.

→ 성별은 이탈에 별로 영향이 크지 않다.